In [1]:
import requests

from bs4 import BeautifulSoup


In [ ]:
#r.content
taxonomy_url = 'https://www.xeno-canto.org/explore/taxonomy'
r = requests.get( taxonomy_url )

soup = BeautifulSoup(r.content, 'html.parser')

#print(soup.prettify()) #print all
type( soup )

In [ ]:
#test going into a sublevel of the taxonomy using combining the 'href' with the original url

ul = soup.find(id='content-area')
#print(ul.prettify())
links = ul.find_all('a')
links_to_store = []
d = {}
for link in links:
    links_to_store.append(link.get('href'))
    #print(link.get('href'))
    
newlink = taxonomy_url + links_to_store[0]
newlink

In [ ]:
#672 total pages
# order by length to speed up time, and exclude unnecessarily long recordings'
#f'https://www.xeno-canto.org/set/5476?pg={page}&dir=0&order=length'
options = '&dir=0&order=length'
#look at page 1 through 400 to start
page = 1
url = f'https://www.xeno-canto.org/set/5476?pg={page}'
url = f'https://www.xeno-canto.org/set/5476?pg={page}&dir=0&order=length'
#url = 'https://www.xeno-canto.org/species/Columba-livia' #rock pidgeon
#url = 'https://www.xeno-canto.org/species/Melanerpes-erythrocephalus' #red headed woodpecker
r = requests.get( url )


In [ ]:
table = soup.find( 'table', {'class', 'results'} )
#table

In [2]:
import os
import sqlite3
from datetime import datetime

def OpenDB(name = 'bird_calls.db'):
    '''open the hippo_stats database for writing'''
    folder = os.getcwd()
    files = [file for file in os.listdir(folder) if file.endswith(".db")]
    #print('cwd', folder)
    print( 'databases found {}'.format(files) )

    if name not in files:
        conn = sqlite3.connect(name)
        c = conn.cursor()
        #create the db if it doesn't exist
        #you can also supply the special name ':memory:' to create a database in RAM

        #create table
        c.execute('''CREATE TABLE bird_calls
                    (id int, common text, scientific text, length text, country text, 
                    location text, elevation int, calltype text, remarks text, rating text, file_path text)'''  )
                     #(date text, time text, IGs real, CGs real)''')
        conn.commit()
        #conn.close()
        #OpenDB()
    else:
        conn = sqlite3.connect(name)
        c = conn.cursor()


    return conn, c


In [ ]:
conn, c = OpenDB()

#id int, common text, scientific text, length text, country text, 
  #location text, elevation int, calltype text, remarks text, rating text, file_path text
testvals = (100, 'common', 'scientific', 'length', 'country',
            'location', 500, 'calltype', 'remarks\nmore remarks', 'rating', 'file_path' )
c.execute("INSERT INTO bird_calls VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", testvals)
conn.commit()
conn.close()

In [3]:
def ProcessEntry( tds, conn, c, D):
    '''take a row from the table and prepare it for entry into the database'''
    id_number = tds[0].find('div', {'class', 'jp-type-single'}).get('data-xc-id')
    names = tds[1].find_all('span') #list == [scientific name, common name]
    try:
        common_name = names[0].text 
        scientific_name = names[1].text
    except:
        print('unknown identity! skipping entry')
        return 0
    length = tds[2].text.strip()  #length of the recording
    contributor = tds[3].text.strip()  #recordist
    date = tds[4].text.strip()  #date of the recording
    time = tds[5].text.strip()  #time of day of the recording
    country = tds[6].text.strip()  #country of recording
    location = tds[7].text.strip()  #location of recording
    elevation = tds[8].text.strip()  #elevation recording occured at
    calltype = tds[9].text.strip()  #type of call, male/female, juvi/adult
    remarks = tds[10].text.strip()
    try:
        rating = tds[11].find('li', {'class', 'selected'} ).text
    except:
        rating = 'F'
    dl_link = tds[11].find_all('a')[0].get('href')
    file_location = f'birdcalls/{common_name}/{id_number}.mp3'
    #only save if A or B grade
    try:
        tempvals = ( int(id_number), common_name, scientific_name, length, country, location, 
                int(elevation), calltype, remarks, rating, file_location )
    except:
        return 0
    #id int, common text, scientific text, length text, country text, 
      #location text, elevation int, calltype text, remarks text, rating text, file_path text
    if rating == 'A' or rating == 'B':
        c.execute("INSERT INTO bird_calls VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", tempvals)
        conn.commit()
        sleep(3)
        D.GetLink( id_number=id_number, folder=common_name, extension=dl_link )
        

#ProcessEntry(tds, conn, c)

In [ ]:
#sample of the loop cell
conn, c = OpenDB()
soup = BeautifulSoup(r.content, 'lxml')#'html.parser')
table = soup.find( 'table', {'class', 'results'} )
headers = table.find_all( 'th' )
for i,head in enumerate(headers):
    print(i, head.text)
#content_area = soup.find( id='content-area') #'table' )
####body = soup.find_all('tr')

#print( f'type {type(body)}, length {len( body )}' )
#print('headers:')
#headers = body[0].find_all( 'th' )

#for i,head in enumerate(headers):
#    print(i, head.text)

#look at page 1 through 400 to start

body = table.find_all('tr')
#print('\nbody:')
for i,tr in enumerate(body[1:-1]):
    #print('i:', i)
    tds = tr.find_all( 'td' )
    ProcessEntry( tds, conn, c, D )



In [ ]:
def GrabOne(c, tograb='311959'):
    t = (tograb,)
    row = c.execute('SELECT * FROM bird_calls WHERE id=?', t)
    #print(c.fetchone())
    return row
    
def GrabAll(c, sortby='length'):
    print('grab all')
    sb = (sortby,)
    rows = c.execute('select * from bird_calls ORDER BY ?', sb)
    return rows

conn, c = OpenDB()
rows = GrabAll(c)
#rows = GrabOne(c, tograb='396222')
for row in rows:
    print(row)

In [4]:
class Downloader(object):
    def __init__(self, main_folder='birdcalls', main_url='https://www.xeno-canto.org'):
        self.main_url = main_url
        self.main_folder = main_folder
        self.path = '/'.join( (os.getcwd(), main_folder) )
        print( 'path =', self.path )
    
    def GetLink(self, id_number, folder, extension):
        '''combine the download extension with the main_url
        check if the species already has a folder, if not create it'''
        
        if folder not in os.listdir(self.path):
            print(f'folder does not exist!  Creating {self.main_folder}/{folder} directory.')
            os.mkdir('/'.join( (self.main_folder, folder) ) )
            
        dl_url = ''.join( (self.main_url, extension) )
        
        r = requests.get( dl_url , allow_redirects=True)
        #open( f'{self.path}/{id_number}.mp3' , 'wb').write(r.content)
        
        print('url', dl_url)
        r = requests.get( dl_url , allow_redirects=True)
        file_location = f'{self.path}/{folder}/{id_number}.mp3'
        open( file_location , 'wb').write(r.content)
        #open( f'{self.path}/{folder}/{id_number}.mp3' , 'wb').write(r.content)
        #return file_location
        
run_test = 0
if run_test:  
    D = Downloader()
    test_id_number= 243410 #375263
    test_dl = f'/{test_id_number}/download'
    D.GetLink( id_number=test_id_number, folder='testfold', extension=test_dl )

In [11]:
#loop over /all/ pages (start with the first few hundred)
#first pass stopped on page 142... (after 3.5 hours) Jan 21, 2020
#second pass start page 142... Jan 22, 2020
#try starting at page 214 to 237
#start 238 to 272 Wednesday morning Jan 22, 2020 (up to 290055, failed on 420402)
#wed night start at page 273 to 312 Jan 22
#Friday Night Jan 24 -- 23:11 start page 313-363
#Saturday Jan 25, pg 363 start, 396 end
#start again and go to page 450 max 397 start, end 428
#start again " start 428 ... none type try 429... failed 430,
#try again, 431?...made it halfway through 458
#Sunday Jan 26, 2020 start at 459 yo 549
#
from time import sleep

conn, c = OpenDB()
D = Downloader()

#run the loop over the pages of interest
for page in range(550, 600):
    print('page number', page)
    #continue
    sleep(0.5)
    url = f'https://www.xeno-canto.org/set/5476?pg={page}&dir=0&order=length'
    r = requests.get( url )
    sleep(1)
    soup = BeautifulSoup( r.content, 'lxml' )
    table = soup.find( 'table', {'class', 'results'} )
    body = table.find_all('tr')
    for i,tr in enumerate(body[1:-1]):
        #print('i:', i)
        tds = tr.find_all( 'td' )
        ProcessEntry( tds, conn, c, D )
        
#else:
#    print(page)

#body = table.find_all('tr')
#print('\nbody:')

databases found ['bird_calls.db']
path = /home/chris/Documents/Insight2020/examples/birdcalls
page number 459
url https://www.xeno-canto.org/135656/download
url https://www.xeno-canto.org/421281/download
url https://www.xeno-canto.org/330579/download
url https://www.xeno-canto.org/244522/download
url https://www.xeno-canto.org/109888/download
url https://www.xeno-canto.org/454252/download
url https://www.xeno-canto.org/318131/download
url https://www.xeno-canto.org/48284/download
url https://www.xeno-canto.org/186483/download
url https://www.xeno-canto.org/263467/download
url https://www.xeno-canto.org/166917/download
url https://www.xeno-canto.org/31287/download
url https://www.xeno-canto.org/381887/download
url https://www.xeno-canto.org/54761/download
url https://www.xeno-canto.org/69941/download
url https://www.xeno-canto.org/133246/download
url https://www.xeno-canto.org/82108/download
url https://www.xeno-canto.org/78652/download
url https://www.xeno-canto.org/317731/download
url

url https://www.xeno-canto.org/78015/download
url https://www.xeno-canto.org/135113/download
url https://www.xeno-canto.org/322017/download
url https://www.xeno-canto.org/364635/download
url https://www.xeno-canto.org/98404/download
url https://www.xeno-canto.org/89176/download
url https://www.xeno-canto.org/89168/download
page number 467
url https://www.xeno-canto.org/391155/download
unknown identity! skipping entry
url https://www.xeno-canto.org/52470/download
url https://www.xeno-canto.org/327671/download
url https://www.xeno-canto.org/193046/download
url https://www.xeno-canto.org/154302/download
url https://www.xeno-canto.org/130977/download
url https://www.xeno-canto.org/416745/download
url https://www.xeno-canto.org/318349/download
url https://www.xeno-canto.org/409009/download
url https://www.xeno-canto.org/420260/download
url https://www.xeno-canto.org/473682/download
url https://www.xeno-canto.org/172533/download
url https://www.xeno-canto.org/138639/download
url https://www.

url https://www.xeno-canto.org/408994/download
url https://www.xeno-canto.org/174765/download
unknown identity! skipping entry
url https://www.xeno-canto.org/179062/download
url https://www.xeno-canto.org/70014/download
page number 475
url https://www.xeno-canto.org/373097/download
url https://www.xeno-canto.org/94546/download
url https://www.xeno-canto.org/189271/download
url https://www.xeno-canto.org/137734/download
url https://www.xeno-canto.org/130549/download
url https://www.xeno-canto.org/331784/download
url https://www.xeno-canto.org/513646/download
url https://www.xeno-canto.org/197272/download
url https://www.xeno-canto.org/356413/download
url https://www.xeno-canto.org/325514/download
url https://www.xeno-canto.org/155232/download
url https://www.xeno-canto.org/182692/download
url https://www.xeno-canto.org/421288/download
url https://www.xeno-canto.org/11857/download
unknown identity! skipping entry
url https://www.xeno-canto.org/101353/download
url https://www.xeno-canto.o

url https://www.xeno-canto.org/173778/download
url https://www.xeno-canto.org/139639/download
url https://www.xeno-canto.org/326381/download
url https://www.xeno-canto.org/308573/download
url https://www.xeno-canto.org/252719/download
url https://www.xeno-canto.org/70471/download
url https://www.xeno-canto.org/16931/download
url https://www.xeno-canto.org/243578/download
page number 483
url https://www.xeno-canto.org/172397/download
url https://www.xeno-canto.org/360825/download
unknown identity! skipping entry
url https://www.xeno-canto.org/138302/download
url https://www.xeno-canto.org/149331/download
url https://www.xeno-canto.org/327791/download
url https://www.xeno-canto.org/191952/download
url https://www.xeno-canto.org/105799/download
url https://www.xeno-canto.org/331563/download
url https://www.xeno-canto.org/90609/download
url https://www.xeno-canto.org/75635/download
url https://www.xeno-canto.org/143701/download
url https://www.xeno-canto.org/205576/download
url https://www

url https://www.xeno-canto.org/382880/download
url https://www.xeno-canto.org/345023/download
url https://www.xeno-canto.org/20530/download
url https://www.xeno-canto.org/175780/download
url https://www.xeno-canto.org/219139/download
url https://www.xeno-canto.org/134515/download
page number 491
url https://www.xeno-canto.org/360564/download
url https://www.xeno-canto.org/142590/download
url https://www.xeno-canto.org/173254/download
url https://www.xeno-canto.org/167821/download
url https://www.xeno-canto.org/252958/download
url https://www.xeno-canto.org/48841/download
url https://www.xeno-canto.org/493601/download
url https://www.xeno-canto.org/70025/download
url https://www.xeno-canto.org/233938/download
url https://www.xeno-canto.org/87304/download
url https://www.xeno-canto.org/328175/download
url https://www.xeno-canto.org/244520/download
url https://www.xeno-canto.org/333311/download
url https://www.xeno-canto.org/137984/download
folder does not exist!  Creating birdcalls/Red-w

...

url https://www.xeno-canto.org/370144/download
url https://www.xeno-canto.org/357733/download
url https://www.xeno-canto.org/478512/download
url https://www.xeno-canto.org/494244/download
url https://www.xeno-canto.org/386075/download
url https://www.xeno-canto.org/507777/download
url https://www.xeno-canto.org/322482/download
url https://www.xeno-canto.org/14893/download
url https://www.xeno-canto.org/373454/download
url https://www.xeno-canto.org/187643/download
url https://www.xeno-canto.org/179321/download
url https://www.xeno-canto.org/144658/download
url https://www.xeno-canto.org/177302/download
url https://www.xeno-canto.org/183317/download
url https://www.xeno-canto.org/392958/download
url https://www.xeno-canto.org/141216/download
url https://www.xeno-canto.org/248008/download
url https://www.xeno-canto.org/131954/download
url https://www.xeno-canto.org/192538/download
url https://www.xeno-canto.org/296901/download
url https://www.xeno-canto.org/254068/download
url https://ww

url https://www.xeno-canto.org/320337/download
url https://www.xeno-canto.org/496432/download
url https://www.xeno-canto.org/180090/download
url https://www.xeno-canto.org/70009/download
url https://www.xeno-canto.org/378994/download
url https://www.xeno-canto.org/317864/download
url https://www.xeno-canto.org/86758/download
url https://www.xeno-canto.org/59190/download
url https://www.xeno-canto.org/187768/download
url https://www.xeno-canto.org/194087/download
url https://www.xeno-canto.org/144691/download
url https://www.xeno-canto.org/508452/download
url https://www.xeno-canto.org/59143/download
url https://www.xeno-canto.org/318883/download
url https://www.xeno-canto.org/456250/download
url https://www.xeno-canto.org/322492/download
url https://www.xeno-canto.org/54750/download
url https://www.xeno-canto.org/101845/download
url https://www.xeno-canto.org/407054/download
url https://www.xeno-canto.org/89717/download
url https://www.xeno-canto.org/136051/download
url https://www.xen

url https://www.xeno-canto.org/137479/download
url https://www.xeno-canto.org/179047/download
url https://www.xeno-canto.org/192705/download
url https://www.xeno-canto.org/412635/download
url https://www.xeno-canto.org/180085/download
url https://www.xeno-canto.org/429248/download
url https://www.xeno-canto.org/469590/download
unknown identity! skipping entry
url https://www.xeno-canto.org/363868/download
url https://www.xeno-canto.org/79481/download
url https://www.xeno-canto.org/105741/download
page number 537
url https://www.xeno-canto.org/233932/download
url https://www.xeno-canto.org/269076/download
url https://www.xeno-canto.org/316284/download
url https://www.xeno-canto.org/106012/download
url https://www.xeno-canto.org/72185/download
url https://www.xeno-canto.org/135055/download
url https://www.xeno-canto.org/381715/download
url https://www.xeno-canto.org/381739/download
url https://www.xeno-canto.org/307349/download
unknown identity! skipping entry
url https://www.xeno-canto.

url https://www.xeno-canto.org/179718/download
url https://www.xeno-canto.org/202181/download
url https://www.xeno-canto.org/78663/download
url https://www.xeno-canto.org/467607/download
page number 544
url https://www.xeno-canto.org/466174/download
url https://www.xeno-canto.org/409007/download
url https://www.xeno-canto.org/316039/download
url https://www.xeno-canto.org/185149/download
url https://www.xeno-canto.org/161091/download
url https://www.xeno-canto.org/309928/download
unknown identity! skipping entry
url https://www.xeno-canto.org/192588/download
url https://www.xeno-canto.org/190465/download
url https://www.xeno-canto.org/487037/download
url https://www.xeno-canto.org/43687/download
url https://www.xeno-canto.org/101826/download
url https://www.xeno-canto.org/165343/download
url https://www.xeno-canto.org/421299/download
url https://www.xeno-canto.org/479492/download
url https://www.xeno-canto.org/130972/download
url https://www.xeno-canto.org/247775/download
url https://w

ConnectionError: HTTPSConnectionPool(host='www.xeno-canto.org', port=443): Max retries exceeded with url: /174838/download (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f0cf7a5a4d0>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [ ]:
#sleep(0.5) #0.5 second pause
print('sleep test')


In [ ]:
common_name = names[0].text
scientific_name = names[1].text
common_name, scientific_name

In [ ]:
location, calltype

In [ ]:
date, time

In [ ]:
id_number = tds[0].find('div', {'class', 'jp-type-single'}).get('data-xc-id')
id_number

In [ ]:
tds[10]
remarks = info_plus_link.find( 'div').text.strip()
#link = info_plus_link.find( 'a')
remarks

In [ ]:
#tds[10].text.strip()
rating = tds[11].find('li', {'class', 'selected'} ).text
rating

In [ ]:
dl_link = tds[11].find_all('a')[0].get('href')
dl_link

In [ ]:
#download the clip at the link
main_url = 'https://www.xeno-canto.org'
dl_url = ''.join( (main_url, dl_link) )
dl_url

dl_r = requests.get( dl_url , allow_redirects=True)
open( f'{id_number}.mp3' , 'wb').write(dl_r.content)

In [ ]:

#testing whole page
info = link_info.find( 'div')
link = link_info.find( 'a')
total_tds = soup.find_all('td')
for k, td in total_tds:
    print(td)


In [ ]:
#alternative to using my 'collection'
location_link = 'https://www.xeno-canto.org/explore?query=box%3A24.28%2C-95.353%2C48.511%2C-61.955%20'
